In [ ]:
!pip install GoogleNews
!pip install gnewsclient
!pip install snscrape==0.6.2.20230320
!pip install rake_nltk

import pandas as pd
# from newsapi import NewsApiClient
import requests
from GoogleNews import GoogleNews
from gnewsclient import gnewsclient
import snscrape.modules.twitter as sntwitter
from datetime import datetime, date
import json
import numpy as np
import pandas as pd
import nltk
import matplotlib
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from string import punctuation
import re
import numpy as np
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from rake_nltk import Rake
from nltk.sentiment.vader import SentimentIntensityAnalyzer as SIA

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 9.8 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.1/81.1 kB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for sgmllib3k: filename=sgmllib3k-1.0.0-py3-none-any.whl size=6046 sha256=dd3e85cad1add82f4c7a24eb147423c8fde9c2c82379720298ea35300da47a8b
  Stored in directory: /root/.cache/pip/wheels/f0/69/93/a47e9d621be168e9e33c7ce60524393c0b92ae83cf6c6e89c5
Successfully built sgmllib3k
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.8/71.8 kB 3.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


/usr/local/lib/python3.10/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [ ]:
start_date = []
end_date = []

In [ ]:
sources = ["bbc-news", "the-telegraph", "the-guardian-uk", "cnn", "abc-news-au",
           "dailymail.co.uk", "metro.co.uk", "mirror.co.uk", "news.google.com"]

In [ ]:
all_keywords = ['strike', 'holiday', 'lockdown',
            'inflation', 'grocery sales', 'carnival', 'festival', 'party', 'Walmart', "Tesco", "Sainsbury's", "supply chain", "flood", "wendys", "lidl"]

# all_keywords = ['tesco', 'holiday']

# all_keywords = ['autumn', 'bank']


In [ ]:
keywords = ['Lidl','Tesco','Walmart','Sainsbury\'s', 'Aldi', 'Asda', 'Marks & Spencers', 'Morrison\'s']

In [ ]:
events = ['autumn bank holiday']

In [ ]:
all_events = ['autumn bank holiday']

final_prod_events = pd.DataFrame()


In [ ]:
counter = 6000

In [ ]:
gnews_client_topics = ['Top Stories',
                       'World',
                       'Nation',
                       'Business',
                       'Technology',
                       'Entertainment',
                       'Sports',
                       'Science',
                       'Health']

In [ ]:
# branch_keyword_bu_num = {'Esher' : 1, 'Dorchester' : 2}
branch_keyword_bu_num = {
'Peterborough': 103,
'Gillingham': 105,
'Dorking': 107,
'St Ives': 108,
'Brighton': 114,
'Brent Cross': 119,
'Dorchester': 120,
'Esher': 121,
'Hall Green': 122,
'Whetstone': 124,
'Coulsdon': 129,
'New Malden': 131,
'Allington Park': 137,
'Bury St Edmunds': 140,
'Blaby': 141,
'Marlow': 146,
'Kingsthorpe': 148,
'East Sheen': 149,
'Four Oaks': 150,
'Westbury Park': 151,
'Leighton Buzzard': 154,
'Stourbridge': 155,
'Bromley': 158,
'Birch Hill': 159,
'Ramsgate': 160,
'Huntingdon': 163,
'Marlborough': 164,
'Green Street Green': 165,
'St Albans': 166,
'Stevenage': 167,
'Havant': 171,
'John Barnes': 174,
'Hertford': 175,
'Beaconsfield': 177,
'Enfield': 179,
'Goldsworth Park': 181,
'Sevenoaks': 182,
'St Neots': 185,
'Ruislip': 197,
'Banstead': 202,
'Ringwood': 203,
'Welwyn Garden City': 204,
'Ely': 205,
'Thame': 206,
'Chichester': 208,
'Southend': 213,
'Henley': 214,
'Finchley': 215,
'Godalming': 216,
'Monmouth': 217,
'Reading': 218,
'Cirencester': 220,
'Berkhamsted': 223,
'Putney': 225,
'Salisbury': 226,
'Billericay': 229,
'Horley': 233,
'Okehampton': 234,
'Waterlooville': 239,
'Biggin Hill': 240,
'Banstead': 324,
'Horsham New': 580,
'Heathfield': 595,
'Cambridge': 651,
'Hailsham': 653,
'Hythe': 654,
'Paddock Wood': 655,
'Saltash': 656,
'Sidmouth': 657,
'Sudbury': 658,
'Thatcham': 659,
'Worcester Park': 661,
'Wymondham': 662,
'Cheltenham': 663,
'Belgravia': 665,
'Tonbridge': 667,
'Chandlers Ford': 668,
'Portishead': 669,
'Romsey': 671,
'Wandsworth': 673,
'Newmarket': 674,
'Sandbach': 680,
'Fulham': 681,
'Towcester': 682,
'Abergavenny': 683,
'Hitchin': 685,
'Swaffham': 686,
'Newport': 687,
'Barry': 688,
'Worthing': 689,
'Otley': 691,
'Farnham': 692,
'Dartford': 693,
'Sheffield': 695,
'Wolverhampton': 696,
'Willerby': 697,
'Lichfield': 699,
'Wilmslow': 711,
'Lewes': 727,
'East Grinstead': 741,
'Buxton': 748,
'St Katharine Docks': 753,
'West Ealing': 764,
'Hersham': 765,
'Bishop s Stortford': 101,
'Buckhurst Hill': 102,
'Epsom': 104,
'Longfield': 109,
'Crowborough': 110,
'Holloway Road': 112,
'Milton Keynes': 115,
'Dibden': 118,
'Burgess Hill': 123,
'Temple Fortune': 125,
'Saffron Walden': 135,
'Evington': 136,
'Witney': 142,
'Harrow Weald': 143,
'Gosport': 152,
'Wantage': 153,
'Daventry': 156,
'Weybridge': 157,
'Winton': 161,
'Andover': 168,
'Southsea': 170,
'Kings Road': 173,
'Cobham': 176,
'Caterham': 178,
'Woodley': 180,
'Harpenden': 183,
'Caversham': 184,
'Northwood': 186,
'Richmond': 188,
'West Byfleet': 189,
'Sunningdale': 190,
'Barnet': 191,
'Chesham': 192,
'Bath': 193,
'Maidenhead': 194,
'Kingston': 195,
'Fleet': 196,
'Yateley': 198,
'Horsham': 200,
'Tenterden': 201,
'Bloomsbury': 207,
'Petersfield': 209,
'Stroud': 210,
'Abingdon': 211,
'Beckenham': 212,
'South Harrow': 219,
'Wokingham': 221,
'Norwich': 222,
'Bromley South': 224,
'Newark': 227,
'Gloucester Road': 230,
'South Woodford': 231,
'Surbiton': 232,
'Staines': 235,
'Marylebone': 236,
'Great Malvern': 237,
'Twyford': 238,
'Byres Road': 308,
'Weston Super Mare': 309,
'Wellington': 315,
'Ashbourne': 316,
'Storrington': 317,
'Menai Bridge': 318,
'Melksham': 319,
'Colchester': 455,
'JL Foodhall Oxford Street': 456,
'Pontprennau': 457,
'Crewkerne': 458,
'Kenilworth': 460,
'Eldon Square': 461,
'Westfield London': 462,
'Winchester': 463,
'Alcester': 474,
'Bridport': 475,
'Caldicot': 476,
'Croydon': 477,
'Haslemere': 478,
'Headington': 479,
'Holsworthy': 480,
'Leigh On Sea': 481,
'Ponteland': 482,
'Saxmundham': 483,
'Stamford': 484,
'Torquay': 485,
'Upminster': 486,
'Lutterworth': 487,
'Clerkenwell': 492,
'JL Foodhall Bluewater': 493,
'Altrincham': 494,
'Frimley': 652,
'Twickenham': 660,
'Canary Wharf': 664,
'Mill Hill': 670,
'Droitwich': 672,
'Wallingford': 675,
'Newbury': 676,
'Sanderstead': 677,
'Kensington': 678,
'Harrogate': 684,
'Rushden': 690,
'Lincoln': 694,
'Rickmansworth': 698,
'Ashford': 705,
'Cheadle Hulme': 710,
'Balham': 719,
'Southampton New': 720,
'Ampthill': 722,
'Durham': 730,
'Barbican': 732,
'Formby': 749,
'Comely Bank': 750,
'Christchurch': 754,
'Bayswater': 756,
'Eastbourne': 757,
'Chiswick': 760,
'Morningside': 761,
'Parkstone': 766,
'Clapham Junction': 767,
'Edgware Road': 768,
'Buckingham': 769,
'Windsor New': 772,
'Islington': 780,
'Hexham': 782,
'Harborne': 796,
'Brackley': 797,
'Lymington New': 798,
'Sandhurst': 799,
'Trinity Square': 833,
'Clifton': 834,
'Crouch End': 835,
'Oxted': 838,
'Enfield CFC': 199,
'Greenford CFC': 259,
'Evesham': 303,
'York': 311,
'Poynton': 312,
'East Cowes': 313,
'Wimbledon': 314,
'Knutsford': 326,
'Newton Mearns': 327,
'Stratford City': 328,
'Alton': 329,
'St Saviour (Jersey)': 332,
'Rohais (Guernsey)': 333,
'St Helier (Jersey)': 334,
'Admiral Park (Guernsey)': 335,
'Red Houses (Jersey)': 336,
'MOUNTSORREL': 403,
'Gerrards Cross': 459,
'Sevenoaks': 464,
'Marlow': 465,
'Cardiff Queen Street': 501,
'Acton': 502,
'Swindon': 504,
'Littlehampton': 505,
'Uckfield': 506,
'Hereford': 507,
'Malmesbury': 511,
'Coulsdon DFC': 513,
'Bagshot': 514,
'Nailsea': 515,
'Parsons Green': 516,
'Egham': 519,
'Jesmond': 520,
'Enfield Chase': 521,
'Sutton Coldfield': 522,
'Chippenham': 523,
'West Hampstead': 524,
'Shrewsbury': 525,
'Tottenham Court Road': 526,
'Dorking': 527,
'Wimbledon Hill': 528,
'Hawkhurst': 529,
'Fulham Palace Road': 530,
'Peterborough': 531,
'Canterbury': 533,
'Sceptre (Watford)': 534,
'Kensington Gardens': 535,
'Camden': 536,
'Addlestone': 542,
'Fitzroy Street': 552,
'Teignmouth': 554,
'Hornchurch': 555,
'Edenbridge': 556,
'Keynsham': 557,
'Spinningfields': 558,
'Cheam': 559,
'Alderley Edge': 560,
'Walton-on-Thames': 562,
'Locks Heath': 563,
'Burgh Heath': 567,
'Petts Wood': 568,
'Portman Square': 569,
'Burnt Common': 571,
'Walbrook': 573,
'Leeds': 574,
'Broxbourne': 575,
'Amersham': 578,
'Bayswater Temp': 579,
'Oxford Botley Road': 581,
'BASINGSTOKE': 582,
'Old Brompton Road': 583,
'Hazlemere': 584,
'Ealing': 586,
'West Kensington': 587,
'Palmers Green': 588,
'Guildford': 589,
'Kings Cross': 590,
'Wollaton': 591,
'Rustington': 596,
'BATTERSEA NINE ELMS': 598,
'UTTOXETER': 599,
'High Holborn': 601,
'Alderley Old': 602,
'Sherborne': 604,
'Hove': 605,
'Leek': 606,
'High Wycombe': 607,
'Hampton': 612,
'Pimlico': 614,
'Foregate Street': 615,
'Clapham Common': 616,
'Kings Cross Station': 619,
'Stirling': 620,
'North Walsham': 622,
'Aylesbury': 625,
'Milngavie': 630,
'Ipswich': 632,
'Manchester Piccadilly': 636,
'Highbury Corner': 637,
'Muswell Hill': 639,
'Knightsbridge': 641,
'Solihull': 642,
'Sidcup': 643,
'Notting Hill Gate': 644,
'Truro': 648,
'Worcester': 700,
'Warminster': 701,
'Exeter': 702,
'South Bank Tower': 703,
'Bracknell': 706,
'Stratford Upon Avon': 708,
'Walton-le-Dale': 721,
'Bedford': 725,
'Wootton': 726,
'Market Harborough': 728,
'Wells': 729,
'Poundbury': 733,
'Cowbridge': 735,
'ROEHAMPTON': 736,
'Battersea': 737,
'Bagshot Road': 738,
'Tubs Hill': 739,
'Greenwich': 740,
'Colmore Row (Birmingham)': 742,
'Ipswich (Corn Exchange)': 743,
'Kings Hill': 744,
'Chipping Sodbury': 751,
'Oakgrove': 752,
'Dorking': 755,
'Oundle': 758,
'Northwich': 759,
'Helensburgh': 771,
'Monument': 773,
'Little Waitrose at John Lewis Watford': 781,
'Victoria Street': 783,
'Vauxhall': 789,
'Horley - Brighton Road': 802,
'Wimborne': 805,
'Headington - London Road': 806,
'Guildford Worplesdon Road': 808,
'Little Waitrose John Lewis Southampton': 815,
'East Putney': 820,
'Meanwood': 828,
'Chester': 842,
'Raynes Park': 846,
'Oadby': 847,
'Leatherhead': 859,
'Victoria Bressenden Place': 860,
'SKY (OSTERLEY)': 865,
'Faringdon': 871,
'Haywards Heath': 873,
'Banbury': 874,
'Finchley Central': 876,
'Bromsgrove': 877,
'Winchmore Hill': 878,
}


In [ ]:
# England = ['Avon', 'Bedfordshire', 'Berkshire', 'Buckinghamshire', 'Cambridgeshire', 'Cheshire', 'Cleveland',
#            'Cornwall', 'Cumbria', 'Derbyshire', 'Devon', 'Dorset', 'Durham', 'East-Sussex', 'Essex', 'Gloucestershire',
#            'Hampshire', 'Herefordshire', 'Hertfordshire', 'Isle-of-Wight', 'Kent', 'Lancashire', 'Leice stershire',
#            'Lincolnshire', 'London', 'Merseyside',
#            'Middlesex', 'Norfolk', 'Northamptonshire', 'Northumberland', 'North-Humberside', 'North-Yorkshire',
#            'Nottinghamshire', 'Oxfordshire', 'Rutland', 'Shropshire', 'Somerset', 'South-Humberside', 'South-Yorkshire',
#            'Staffordshire', 'Suffolk', 'Surrey', 'Tyne-and-Wear', 'Warwickshire', 'West-Midlands', 'West-Sussex',
#            'West-Yorkshire', 'Wiltshire', 'Worcestershire']
England = ['London']
Wales = ['Clwyd', 'Dyfed', 'Gwent', 'Gwynedd', 'Mid-Glamorgan',
         'Powys', 'South-Glamorgan', 'West-Glamorgan']
# Wales = ['South-Glamorgan']
Scotland = ['Aberdeenshire', 'Angus', 'Argyll', 'Ayrshire', 'Banffshire', 'Berwickshire', 'Bute', 'Caithness',
            'Clackmannanshire', 'Dumfriesshire', 'Dunbartonshire', 'East-Lothian', 'Fife', 'Inverness-shire',
            'Kincardineshire', 'Kinross-shire',
            'Kirkcudbrightshire', 'Lanarkshire', 'Midlothian', 'Moray', 'Nairnshire', 'Orkney', 'Peeblesshire',
            'Perthshire', 'Renfrewshire', 'Ross-shire', 'Roxburghshire', 'Selkirkshire', 'Shetland', 'Stirlingshire',
            'Sutherland', 'West Lothian', 'Wigtownshire']
NorthernIreland = ['Antrim', 'Armagh', 'Down',
                   'Fermanagh', 'Londonderry', 'Tyrone']

In [ ]:
# branch_keyword = ['Abergavenny', 'Alderley Edge', "Eastbourne", "Edenbridge", "Pontprennau"]
# branch_keyword = ['Abingdon', 'Canary Wharf']
# all_branch_keyword = ['Yateley', 'Canary Wharf', 'Workingham', 'Firmley']
all_branch_keyword = list(branch_keyword_bu_num.keys())

In [ ]:
branch_keyword = all_branch_keyword[:3]

In [ ]:
# countries = [England, Wales, Scotland, NorthernIreland]
countries = [England]

In [ ]:
final = []

In [ ]:
# final_prod = pd.DataFrame()
status_val = []

In [ ]:
def googleNewsByStreet():
    data = pd.DataFrame()
    for branch in branch_keyword:
        for keyword in keywords:
            news = GoogleNews()
            news.set_period('1d')
            news.get_news(branch + ' ' + keyword)
            results = news.result()
            df = pd.DataFrame.from_dict(results)
            df['keyword'] = keyword
            df['branch'] = branch
            df['bu_num'] = branch_keyword_bu_num[branch]
            print(df)
            df.head(5)
            data = data.append(df, ignore_index=True)

    # print the dataframe
    if len(data.columns) > 4:
      data = data.drop(columns=["img", "site"])
      final.append(data)


In [ ]:
def _removeNonAscii(s):
    return "".join(i for i in s if ord(i) < 128)

function to remove the punctuations, apostrophe, special characters using regular expressions

In [ ]:
def clean_text(text):
    text = text.lower()
    text = re.sub(r"what's", "what is ", text)
    text = text.replace('(ap)', '')
    text = re.sub(r"\'s", " is ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"can't", "cannot ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r'\W+', ' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r"\\", "", text)
    text = re.sub(r"\'", "", text)
    text = re.sub(r"\"", "", text)
    text = re.sub('[^a-zA-Z ?!]+', '', text)
    text = _removeNonAscii(text)
    text = text.strip()
    return text

stop words are the words that convery little to no information about the actual content like the words:the, of, for etc

In [ ]:
def remove_stopwords(word_tokens):
    filtered_sentence = []
    stop_words = stopwords.words('english')
    specific_words_list = ['char', 'u', 'hindustan', 'doj', 'washington']
    stop_words.extend(specific_words_list)
    for w in word_tokens:
        if w not in stop_words:
            filtered_sentence.append(w)
    return filtered_sentence

function for lemmatization

In [ ]:
def lemmatize(x):
    lemmatizer = WordNetLemmatizer()
    return ' '.join([lemmatizer.lemmatize(word) for word in x])

splitting a string, text into a list of tokens

In [ ]:
tokenizer = RegexpTokenizer(r'\w+')

In [ ]:
def tokenize(x):
    return tokenizer.tokenize(x)

In [ ]:
nltk.download('all')

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [ ]:
def sentiment_analysis(prod):
    prod['combined_text'] = prod['title'].map(str)

    # applying all of these functions to the our dataframe 
    prod['combined_text'] = prod['combined_text'].map(clean_text)
    prod['tokens'] = prod['combined_text'].map(tokenize)
    prod['tokens'] = prod['tokens'].map(remove_stopwords)
    prod['lems'] = prod['tokens'].map(lemmatize)
    sia = SIA()
    results = []
    for line in prod['lems']:
        pol_score = sia.polarity_scores(line)
        pol_score['lems'] = line
        results.append(pol_score)
    headlines_polarity = pd.DataFrame.from_records(results)
    temp = []
    # for line in prod['branch']:
        # temp.append(line)
    # headlines_polarity['branch'] = temp
    headlines_polarity['label'] = 0
    headlines_polarity.loc[headlines_polarity['compound'] > 0.2, 'label'] = 1
    headlines_polarity.loc[headlines_polarity['compound'] < -0.2, 'label'] = -1
    headlines_polarity['word_count'] = headlines_polarity['lems'].apply(lambda x: len(str(x).split()))
    headlines_polarity.head()
    # gk = headlines_polarity.groupby(['branch', 'label'])
    # fk = headlines_polarity.groupby('branch')['compound'].mean()
    # fk = fk.to_frame()
    result = [prod, headlines_polarity]
    headlines_polarity = headlines_polarity.rename_axis(index=None)
    return pd.merge(prod, headlines_polarity, on=["lems"], how="left")
    

In [ ]:
from datetime import date

def outsource_news():
    googleNewsByStreet()
    prod = pd.concat(final)
    prod = prod.drop_duplicates('title', keep='first')
    print(prod)
    status_val.append(30)

    final_prod = sentiment_analysis(prod)

    upload_data_complete(final_prod)
      
    final_prod = final_prod.replace(np.nan,'',regex=True)

    # forecast_keywords = ['sale', 'sport', 'beverage', 'retail', 'vendor', 'market', 'morrisons', 'tesco', 'coles', 'business', 'shopping', 'weather', 
    #                      'parties', 'events', 'walmart']

    second_keywords = ['bank holiday', 'heatwave', 'inflation', 'street party', 'rainfall', 'snow', 'retail', 'beverage', 'tesco', 'walmart', 'morrisons', 'weather', 
                       'brc', 'mothers day', 'new store launch', 'lidl', 'homebase', 'walmart', 'new tesco store', 'coles', 'supermarket', 'shoppers', 'store', 'grocery', 'strike', 'holiday'
                       'shops', 'markets','holiday', 'lockdown','grocery sales', 'carnival', 'festival', 'party', "sainsbury", "supply chain", "flood", "wendys",
                       'ocado', 'spencer', 'asda']                    

    remove_keywords = ['accident', 'incident', 'injury', 'political', 'police', 'death', 'traffic', 'lord', 'war', 'actor', 'movie', 'star', 'lord', 'sex', 'gay',
                       'fight', 'crash', 'life', 'weapons', 'dating', 'radio', 'tv', 'guinness', 'husband', 'fashion', 'attack']

    store_keywords = ['opens', 'closes', 'closed', 'opened', 'open', 'close', 
                      'shut', 'confining', 'unopen', 'opening',
                      'close down', 'closing', 'shut down', 'conclude', 'ending', 'shutdown', 'closedown',
                      'closure', 'temporary', 'extended', 'shutting', 'launch', 'shuts', 'closures']

    store_remove_keywords = ['ftse', 'pubs', 'pub', 'life', 'stocks', 'earnings', 'dining', 'restaurants', 'stock', 'rocket', 'fashion', 'restaurant',
                             'letter', 'bills', 'investment', 'childrenswear', 'blizzard', 'infamous', 'qualifying', 'sports', 'bar', 'cafe', 
                             'technology', 'dental', 'boobs', 'school', 'flixbus', 'allegations', 'pharmacy', 'attack', 'driver', 'fitness', 'students',
                             'charities']
                  
    competitor_keywords = ['tesco', 'wendys', 'lidl', 'sainsburys', 'sainsbury', 'aldi', 'morrisons', 'spencer', 'asda', 'supermarket',
                            'co', 'ocado', 'sparks', 'b&m', 'iceland', 'waitrose']

    print(final_prod)

    for index, row in final_prod.iterrows():
      if (len(np.intersect1d(row['tokens'], store_keywords)) == 0):
        # if(len(np.intersect1d(row['tokens'], competitor_keywords)) == 0):
        final_prod.drop(index=index, axis=0, inplace=True)
      else:
        if(len(np.intersect1d(row['tokens'], competitor_keywords)) == 0):
          final_prod.drop(index=index, axis=0, inplace=True)

    for index, row in final_prod.iterrows():
      for value in row['tokens']:
        val = value.capitalize()
        try:
            final_prod.at[index,'bu_num'] = branch_keyword_bu_num[val]
            final_prod.at[index,'branch'] = val
        except:
            n = 0

    final_prod = final_prod.drop_duplicates('title', keep='first')
    final_prod = final_prod.drop_duplicates('lems', keep='first')
    final_prod = final_prod.drop_duplicates('tokens', keep='first')

    final_prod['title'] = final_prod['title'].astype(str)

    final_prod['competitor_evt_indchar'] = ['Yes' if(len(np.intersect1d(x,competitor_keywords)) > 0) else 'No' for x in final_prod['tokens']]

    counter_guid = int(date.today().strftime("%Y%m%d"))
    final_prod['efsevt_guid'] = [(counter_guid*1000)+i for i in range(len(final_prod))]

    print(final_prod.dtypes)
    print(final_prod_events.dtypes)

    foriegn_key = []

    for index, row in final_prod.iterrows():
      flag = False
      for index_event, row_event in final_prod_events.iterrows():
        if(row['keyword'] != '' ):
          if(row['keyword'] in row_event['NAME']):
            print(row['keyword'],row_event['NAME'])
            foriegn_key.append(row_event['GUID'])
            flag = True
      if(flag == False):
        foriegn_key.append(0)

    print(foriegn_key)

    final_prod['guid'] = [(counter_guid*2000)+i for i in range(len(final_prod))]
    final_prod['fixed_annual_ind'] = 'n'
    final_prod['perm_env_ind'] = 'n'
    final_prod['cancelled_ind'] = 'n'
    final_prod['create_user'] = ''
    final_prod['update_user'] = ''
    final_prod['perm_env_ind'] = 'n'    
    final_prod['crt_timestamp'] = date.today()
    final_prod['upd_timestamp'] = date.today()

    final_prod.rename(columns = {'link':'source_of_event'}, inplace = True)

    final_prod[["datetime"]] = final_prod[["datetime"]].astype(str)
    distance_arr = []
    for index, row in final_prod.iterrows():
      if(row["KEYWORD"] and row["BRANCH"]):
          distance_arr.append(place_distance(row["KEYWORD"], row["BRANCH"]))
    final_prod["DISTANCE IN MILES"] = distance_arr

    final_prod.columns = final_prod.columns.str.upper()

    distance_arr = []
    for index, row in final_prod.iterrows():
        if(row["KEYWORD"] and row["BRANCH"]):
            distance_arr.append(place_distance(row["KEYWORD"], row["BRANCH"]))

    final_prod["DISTANCE"] = distance_arr

    upload_data(final_prod)
    return final_prod


In [ ]:
from google.colab import auth
auth.authenticate_user()
import pandas as pd
from gspread_dataframe import set_with_dataframe
import gspread
from google.auth import default
creds, _ = default()

from google.colab import drive

def upload_data(final_prod):
  # final_prod = outsource_filtered_news()
  # final_prod = outsource_news()
  final_prod = final_prod.drop_duplicates('LEMS', keep='first')

  print("Entered", len(final_prod))
  status_val.append(50)
  final_prod = final_prod.append([], ignore_index=True)
  gc = gspread.authorize(creds)
  sh = gc.create(date.today().strftime("%d/%m/%Y"))
  worksheet = gc.open(date.today().strftime("%d/%m/%Y")).sheet1
  set_with_dataframe(worksheet, final_prod)

  gc3 = gspread.authorize(creds)
  sh = gc3.create(date.today().strftime("%d/%m/%Y") + '_event_occurence')
  worksheet3 = gc3.open(date.today().strftime("%d/%m/%Y") + '_event_occurence').sheet1
  final_prod_occurence = pd.DataFrame()
  final_prod_occurence['GUID'] = final_prod['GUID']
  final_prod_occurence['SDATE'] = final_prod['DATETIME']
  final_prod_occurence['EDATE'] = final_prod['DATETIME']
  final_prod_occurence['FIXED_ANNUAL_IND'] = final_prod['FIXED_ANNUAL_IND']
  final_prod_occurence['PERM_ENV_IND'] = final_prod['PERM_ENV_IND']
  final_prod_occurence['CANCELLED_IND'] = final_prod['CANCELLED_IND']
  final_prod_occurence['CREATE_USER'] = final_prod['CREATE_USER']
  final_prod_occurence['UPDATE_USER'] = final_prod['UPDATE_USER']
  final_prod_occurence['CRT_TIMESTAMP'] = final_prod['CRT_TIMESTAMP']
  final_prod_occurence['UPD_TIMESTAMP'] = final_prod['UPD_TIMESTAMP']
  final_prod_occurence['EFSEVT_GUID'] = final_prod['EFSEVT_GUID']
  final_prod_occurence['SOURCE_OF_EVENT'] = final_prod['SOURCE_OF_EVENT']
  final_prod_occurence['COMMENTS'] = " "
  set_with_dataframe(worksheet3, final_prod_occurence)


  # for i in range(len())
  gc1 = gspread.authorize(creds)
  sh1 = gc1.create(date.today().strftime("%d/%m/%Y") + '_event_type')
  worksheet1 = gc1.open(date.today().strftime("%d/%m/%Y") + '_event_type').sheet1
  final_prod_events['GUID'] = final_prod['EFSEVT_GUID']
  final_prod_events['NAME'] = final_prod['TITLE']
  final_prod_events['CREATE_USER'] = ' '
  final_prod_events['UPDATE_USER'] = ' '
  final_prod_events['INCLUDED_IND'] = 'y'
  final_prod_events['COMPETITOR_EVT_INDCHAR'] = final_prod['COMPETITOR_EVT_INDCHAR']
  # final_prod_events['NAME_TOKENS'] = events.map(remove_stopwords)
  # print(final_prod_events['NAME_TOKENS'])
  final_prod_events['OSO_NAME'] = final_prod['TITLE']
  final_prod_events['CRT_TIMESTAMP'] = ''
  final_prod_events['UPD_TIMESTAMP'] = ''
  set_with_dataframe(worksheet1, final_prod_events)

  gc2 = gspread.authorize(creds)
  sh2 = gc2.create(date.today().strftime("%d/%m/%Y") + '_E2B')
  worksheet2 = gc2.open(date.today().strftime("%d/%m/%Y") + '_E2B').sheet1
  try:
    e2b_dataframe = final_prod[['EFSEVT_GUID', 'BU_NUM']].copy()
  except:
    e2b_dataframe = final_prod[['EFSEVT_GUID']].copy()
  e2b_dataframe.dropna()
  set_with_dataframe(worksheet2, e2b_dataframe)



  dates = datetime.now()
  dates_E2B = date.today().strftime("%d %m %Y_E2B") + '.gsheet'
  dates_type = date.today().strftime("%d %m %Y_event_type") + '.gsheet'
  dates_occurence = date.today().strftime("%d %m %Y_event_occurence") + '.gsheet'
  print(dates)

  drive.mount('/content/drive/', force_remount=True)  
  !ls -tl /content/drive/My\ Drive/ | head -5
  !mkdir /content/drive/My\ Drive/Reports/'{dates}'
  !mv /content/drive/My\ Drive/'{dates_E2B}' /content/drive/My\ Drive/Reports/'{dates}'
  !mv /content/drive/My\ Drive/'{dates_type}' /content/drive/My\ Drive/Reports/'{dates}'
  !mv /content/drive/My\ Drive/'{dates_occurence}' /content/drive/My\ Drive/Reports/'{dates}'

  return final_prod


In [ ]:
from google.colab import auth
auth.authenticate_user()
from gspread_dataframe import set_with_dataframe
import gspread
from google.auth import default
creds, _ = default()

def upload_data_complete(final_prod):
  gc = gspread.authorize(creds)
  sh = gc.create(date.today().strftime("%d/%m/%Y") + '_complete')
  worksheet = gc.open(date.today().strftime("%d/%m/%Y") + '_complete').sheet1
  set_with_dataframe(worksheet, final_prod)

In [ ]:
!pip install geopy
!pip install pgeocode

from geopy.geocoders import Photon, GoogleV3, Nominatim
import pgeocode
from math import cos, asin, sqrt, pi

def distance(lat1, lon1, lat2, lon2):
    p = pi/180
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p) * cos(lat2*p) * (1-cos((lon2-lon1)*p))/2
    return round(12742 * asin(sqrt(a)),2)


def place_distance(string1,string2):
    try:
      geolocator_addr = Nominatim(user_agent="http")
      # place = "Lidl,Bath"
      # place_2 = "Waitrose,Bath"
      place = string1 + "," + string2
      place_2 = "Waitrose," + string2
      # location = geolocator.geocode(place)
      pin = geolocator_addr.geocode(place)
      pin_2 = geolocator_addr.geocode(place_2)
      # print(location)
      print(pin)
      print(pin_2)
      print(pin.raw['lat'],pin.raw['lon'],pin_2.raw['lat'],pin_2.raw['lon'])
    except:
      return 0
    
    return distance(float(pin.raw['lat']),float(pin.raw['lon']),float(pin_2.raw['lat']),float(pin_2.raw['lon']))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pretty-html-table
!pip install pyshorteners

import smtplib, ssl
from smtplib import SMTP
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
import pandas as pd
from pretty_html_table import build_table
from pyshorteners import Shortener

def mail_data(final_prod):
  port = 465  # For SSL
  context = ssl.create_default_context()
  
  mail_df = pd.DataFrame()
  mail_df["TITLE"] = final_prod["TITLE"]
  mail_df["BRANCH"] = final_prod["BRANCH"]
  mail_df["SOURCE"] = final_prod["MEDIA"]
  mail_df["DATETIME"] = final_prod["DATETIME"]
  mail_df["BRANCH_NUM"] = final_prod["BU_NUM"]

  distance_arr = []
  for index, row in final_prod.iterrows():
    if(row["KEYWORD"] and row["BRANCH"]):
        distance_arr.append(place_distance(row["KEYWORD"], row["BRANCH"]))
  mail_df["DISTANCE IN MILES"] = distance_arr
  
  urls = []
  for index,row in final_prod.iterrows(): 
      x = Shortener().tinyurl.short(row["SOURCE_OF_EVENT"])
      urls.append(x)
  
  mail_df["LINK"] = urls

  html_table = mail_df.to_html(index=False, classes='example-table')

  text = f"Hello Alex and Olga,\n Herewith attaching the events captured for all the competitors (core event types) including all the branches from 25-05-2023 to 26-05-2023 which are auto-generated from the script.\n\n\nThanks And Regards,\nSashank L\n\n\n"


  print(mail_df)

  html_table = html_table.replace('<th>', '<th style="padding: 10px 90px 10px 90px;">', 1)

# HTML Styling
  html = f'''
<html>
<head>
    <style>
        table.example-table th{{
              padding: 10px;
              text-align: center;
              background-color: #FFFFFF;
              font-weight: bold;
              font-size: 14px;
              width: 400px;
          }}

          table.example-table th:first-child {{
            padding: 20px 100px 20px 100px; /* Set the desired width for the fourth column */
          }}

          table.example-table td {{
            padding: 5px;
            color: black;
            font-size: 12px;
            width: 400px;
            font-family: Century Gothic, sans-serif;
          }}

        /* Add custom styles here */
    </style>
</head>
<body>
      <pre>{text}</pre>
        {html_table}
</body>
</html>
'''

  part1 = MIMEText(html, 'html')
  msg = MIMEMultipart("alternative")
  msg['Subject'] = "Automated Event Capturing Model"
  # recipients = ['amit.kumbhar@johnlewis.co.uk', 'mitali.patel@johnlewis.co.uk', "alex.nicola@waitrose.co.uk", "olga.jakobsone@waitrose.co.uk"]
  recipients = ['mitali.patel@johnlewis.co.uk']
  msg['To'] = ", ".join(recipients)
  msg.attach(part1)

  with smtplib.SMTP_SSL("smtp.gmail.com", port, context=context) as server:
      print("IN")
      server.login("sashank.lakshminarayan@johnlewis.co.uk", "xnkefnwjkmjoevav")
      server.sendmail("sashank.lakshminarayan@johnlewis.co.uk", recipients, msg.as_string())


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Preparing metadata (setup.py) ... done
  Created wheel for pyshorteners: filename=pyshorteners-1.0.1-py3-none-any.whl size=17479 sha256=5f6a9eafb2c4dd8c65fdd577cf371d106f6f91c0a7c18bfa95296a5136ba17d9
  Stored in directory: /root/.cache/pip/wheels/d5/ff/b3/16d8906f92ab0b042f76a4696fcea11d291d79a2b3a3de1e78
Successfully built pyshorteners


In [ ]:
if __name__ == '__main__':
  final_prod = outsource_news()
  mail_data(final_prod)

Empty DataFrame
Columns: [keyword, branch, bu_num]
Index: []


<ipython-input-17-e8bb4f847922>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df, ignore_index=True)


                                               title  desc          date  \
0  Seven important changes to money, benefits and...  None  20 hours ago   

                    datetime  \
0 2023-05-30 06:37:44.246899   

                                                link  \
0  news.google.com/./articles/CBMiZGh0dHBzOi8vd3d...   

                                                 img                media  \
0  https://lh3.googleusercontent.com/kxPIY-2LgfrH...  Cambridgeshire Live   

   site keyword        branch  bu_num  
0  None   Tesco  Peterborough     531  


<ipython-input-17-e8bb4f847922>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df, ignore_index=True)


Empty DataFrame
Columns: [keyword, branch, bu_num]
Index: []


...

<ipython-input-17-e8bb4f847922>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df, ignore_index=True)


Empty DataFrame
Columns: [keyword, branch, bu_num]
Index: []


<ipython-input-17-e8bb4f847922>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df, ignore_index=True)


Empty DataFrame
Columns: [keyword, branch, bu_num]
Index: []


<ipython-input-17-e8bb4f847922>:15: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  data = data.append(df, ignore_index=True)


In [ ]:
from datetime import date
from google.colab import auth
auth.authenticate_user()
from google.auth import default
creds, _ = default()
from gspread_dataframe import set_with_dataframe
import gspread
import pandas as pd
from flask import Flask, jsonify, Response, request
# from flask_ngrok import run_with_ngrok
from json import loads

def validate():
    gc = gspread.authorize(creds)
    worksheet = gc.open("26/05/2023").sheet1
    final_prod = pd.DataFrame(data=worksheet.get_all_records())
    mail_data(final_prod)
    # print(final_prod.to_json(orient="index"))
    # return Response(final_prod.to_json(orient="index"), mimetype='application/json')

# validate()